In [ ]:
import polars as pl

In [ ]:
#Question 1a
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/Colab Notebooks/economics_long.csv"

# Load the data from the CSV file into a Polars DataFrame
economics_long = pl.read_csv(file_path)

economics_wide = economics_long.pivot(
    values="value",
    index="date",
    on="variable"
)

economics_wide

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


date,pce,pop,psavert,uempmed,unemploy
str,f64,f64,f64,f64,f64
"""1967-07-01""",506.7,198712.0,12.6,4.5,2944.0
"""1967-08-01""",509.8,198911.0,12.6,4.7,2945.0
"""1967-09-01""",515.6,199113.0,11.9,4.6,2958.0
"""1967-10-01""",512.2,199311.0,12.9,4.9,3143.0
"""1967-11-01""",517.4,199498.0,12.8,4.7,3066.0
…,…,…,…,…,…
"""2014-12-01""",12062.0,319746.157,7.6,12.9,8717.0
"""2015-01-01""",12046.0,319928.646,7.7,13.2,8903.0
"""2015-02-01""",12082.4,320074.511,7.9,12.9,8610.0


In [ ]:
#Question 1b
median_unemploy = (
    economics_wide
    .filter(pl.col("date") >= "2010-01-01")
    .select(pl.col("unemploy").median())
    .item()
)

print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")

The median unemployment for 2010-01-01 and beyond is: 12471.0


In [25]:
#Question 2a
from google.colab import drive
import polars as pl
import io



file_path = "/content/drive/MyDrive/Colab Notebooks/AirPassengers.txt"


with open(file_path, 'r') as f:
    file_content = f.read()


lines = file_content.strip().split('\n')
header = lines[0].split()
data_rows = [line.split() for line in lines[1:]]


data = []
for row in data_rows:
    row_dict = {header[i]: row[i] for i in range(len(header))}
    data.append(row_dict)


air = pl.DataFrame(data)


if "Year" in air.columns and "year" not in air.columns:
    air = air.rename({"Year": "year"})


air_passengers_long = air.unpivot(
    index="year",
    on=[c for c in air.columns if c != "year"],
    variable_name="month",
    value_name="passengers"
).with_columns(
    pl.col("passengers").cast(pl.Int64)
)


print("Original shape:", air.shape)
print("Long shape:", air_passengers_long.shape)
print(air_passengers_long.head())


air_passengers_long

Original shape: (12, 13)
Long shape: (144, 3)
shape: (5, 3)
┌──────┬───────┬────────────┐
│ year ┆ month ┆ passengers │
│ ---  ┆ ---   ┆ ---        │
│ str  ┆ str   ┆ i64        │
╞══════╪═══════╪════════════╡
│ 1949 ┆ Jan   ┆ 112        │
│ 1950 ┆ Jan   ┆ 115        │
│ 1951 ┆ Jan   ┆ 145        │
│ 1952 ┆ Jan   ┆ 171        │
│ 1953 ┆ Jan   ┆ 196        │
└──────┴───────┴────────────┘


year,month,passengers
str,str,i64
"""1949""","""Jan""",112
"""1950""","""Jan""",115
"""1951""","""Jan""",145
"""1952""","""Jan""",171
"""1953""","""Jan""",196
…,…,…
"""1956""","""Dec""",306
"""1957""","""Dec""",336
"""1958""","""Dec""",337


In [24]:
#Question 2b
std_passengers = (
    air_passengers_long
    .filter((pl.col("year").cast(pl.Int64) >= 1955) & (pl.col("year").cast(pl.Int64) <= 1960))
    .select(pl.col("passengers").std())
    .item()
)

print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers:.2f}")

The standard deviation of airline passengers between 1955 and 1960 is: 86.44


In [26]:
#Question 3a
from google.colab import drive

file_path = "/content/drive/MyDrive/Colab Notebooks/table2.csv"


table2 = pl.read_csv(file_path)

print("Original data:")
print(table2.head())

table2_wide = table2.pivot(
    values="count",
    index=["country", "year"],
    columns="type"
)

table2_wide

Original data:
shape: (5, 5)
┌──────────┬─────────────┬──────┬────────────┬──────────┐
│ rownames ┆ country     ┆ year ┆ type       ┆ count    │
│ ---      ┆ ---         ┆ ---  ┆ ---        ┆ ---      │
│ i64      ┆ str         ┆ i64  ┆ str        ┆ i64      │
╞══════════╪═════════════╪══════╪════════════╪══════════╡
│ 1        ┆ Afghanistan ┆ 1999 ┆ cases      ┆ 745      │
│ 2        ┆ Afghanistan ┆ 1999 ┆ population ┆ 19987071 │
│ 3        ┆ Afghanistan ┆ 2000 ┆ cases      ┆ 2666     │
│ 4        ┆ Afghanistan ┆ 2000 ┆ population ┆ 20595360 │
│ 5        ┆ Brazil      ┆ 1999 ┆ cases      ┆ 37737    │
└──────────┴─────────────┴──────┴────────────┴──────────┘


/tmp/ipython-input-1824952141.py:12: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  table2_wide = table2.pivot(


country,year,cases,population
str,i64,i64,i64
"""Afghanistan""",1999,745,19987071
"""Afghanistan""",2000,2666,20595360
"""Brazil""",1999,37737,172006362
"""Brazil""",2000,80488,174504898
"""China""",1999,212258,1272915272
"""China""",2000,213766,1280428583


In [29]:
#Question 3b
table2_with_ratio = table2_wide.with_columns(
    (pl.col("cases") / pl.col("population")).alias("ratio")
)

# 2. Filter for year = 1999
year_1999 = table2_with_ratio.filter(pl.col("year") == 1999)

# 3. Find the country with the highest ratio
highest_ratio_country = (
    year_1999.sort("ratio", descending=True)
    .select("country")
    .head(1)  # Get the first row as a DataFrame
    .item()   # Extract the scalar value from the single-cell DataFrame
)

print("Data for 1999 with ratios:")
print(year_1999)

print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬────────┬────────────┬──────────┐
│ country     ┆ year ┆ cases  ┆ population ┆ ratio    │
│ ---         ┆ ---  ┆ ---    ┆ ---        ┆ ---      │
│ str         ┆ i64  ┆ i64    ┆ i64        ┆ f64      │
╞═════════════╪══════╪════════╪════════════╪══════════╡
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   ┆ 0.000037 │
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 ┆ 0.000167 │
└─────────────┴──────┴────────┴────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: Brazil
